This cell is for loading the data and installing relevant librarys

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


file = ('/Users/olisa/Downloads/clean_df.csv') # convert to website link later
data = pd.read_csv(file)

0     $50.00
1     $75.00
2     $90.00
3     $55.00
4    $379.00
Name: price, dtype: object

In [3]:
print(data.columns)

Index(['id', 'listing_url', 'name', 'description', 'neighborhood_overview',
       'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_picture_url',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms_text', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'number_of_reviews',
       'number_of_reviews_ltm', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'ca

Next step is cleaning the dataframe and removing the colums that are not neccsary for the analysis.

In [4]:

relevant_columns = ['price', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location','review_scores_value']
subset_data = data[relevant_columns]

# Display the first 5 rows of the new DataFrame
print(subset_data.head())



     price  review_scores_rating  review_scores_accuracy  \
0   $50.00                  4.90                    4.82   
1   $75.00                  4.79                    4.84   
2   $90.00                  4.32                    4.53   
3   $55.00                  4.84                    4.91   
4  $379.00                  4.74                    4.82   

   review_scores_cleanliness  review_scores_checkin  \
0                       4.89                   4.86   
1                       4.88                   4.87   
2                       4.03                   4.72   
3                       4.71                   4.93   
4                       4.69                   4.69   

   review_scores_communication  review_scores_location  review_scores_value  
0                         4.93                    4.75                 4.82  
1                         4.82                    4.93                 4.73  
2                         4.86                    4.72                 4.3

Now lets check and potentially handle any missing values and convert any datatypes if needed

In [27]:
print(subset_data.dtypes)
print(subset_data.isna().sum(axis=0))

price                          float64
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
review_scores_value            float64
dtype: object
price                              0
review_scores_rating           16785
review_scores_accuracy         17807
review_scores_cleanliness      17794
review_scores_checkin          17841
review_scores_communication    17808
review_scores_location         17839
review_scores_value            17842
dtype: int64


We have confirmed that there are some missing values so we will need to fix that. In addition price is in the wrong type so lets convert it to a neumeric one. After that we will proceed with some Descriptive statistics

In [28]:
# As there are a significant amount of missing values for the review score we will have to 'drop' thiese rows from the dataframe entirely

subset_data = subset_data.dropna()

In [30]:
# Converting datatype for 'price' 
subset_data['price'] = subset_data['price'].astype(str).str.replace('$', '').astype(float)

print(subset_data['price'])
print(subset_data['price'].dtypes)


0         50.0
1         75.0
2         90.0
3         55.0
4        379.0
         ...  
69346     55.0
69347    201.0
69348    246.0
69349    250.0
69350    134.0
Name: price, Length: 51505, dtype: float64
float64


In [40]:
# Mean
mean_price = subset_data['price'].mean()

# Median
median_price = subset_data['price'].median()

# Standard Deviation
std_price = subset_data['price'].std()

# Minimum and Maximum
min_price = subset_data['price'].min()
max_price = subset_data['price'].max()

# Quartiles 
q25_price = subset_data['price'].quantile(0.25)
q75_price = subset_data['price'].quantile(0.75)

i = 0
summ_stats = [('mean = ', mean_price), ('median = ', median_price),('standard deviation = ', std_price),('min = ' , min_price), ('max = ' , max_price), ('q25 = ' , q25_price), ('q25 = ', q75_price)]
while(i < 7):
    print(summ_stats[i])
    i += 1

('mean = ', 134.02898184642268)
('median = ', 96.0)
('standard deviation = ', 125.55085686084625)
('min = ', 0.0)
('max = ', 999.0)
('q25 = ', 55.0)
('q25 = ', 165.0)


these stats on there own are not very useful so lets do some more work

# now we are going to start with data visualisation

# Boxplot - A boxplot provides a concise summary of the distribution of the data, including measures such as the median, quartiles, and potential outliers. Here's how you can create a boxplot for the 'price' variable:

In [32]:

#boxplot of price distribution

plt.figure(figsize=(8, 6))
sns.boxplot(x='price', data=subset_data)
plt.title('Boxplot of Price')
plt.xlabel('Price')
plt.show()


/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/1660440810.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [42]:
# Boxplots comparing each review score with the listing price

plt.figure(figsize=(10, 6))

review_scores = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin']

plt.figure(figsize=(10, 6))

review_scores = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin']

for score in review_scores:
    sns.boxplot(x=score, y='price', data=subset_data, palette='pastel')
    plt.xlabel(f'Review Score ({score.replace("review_scores_", "")})')
    plt.ylabel('Listing Price')
    plt.title(f'Impact of {score} on Listing Price')
    plt.tight_layout()
    plt.show()

/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/3901070994.py:12: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x=score, y='price', data=subset_data, palette='pastel')
/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/3901070994.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/3901070994.py:12: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x=score, y='price', data=subset_data, palette='pastel')
/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/3901070994.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
 

Scatter plots vizualize the relationship between review score and price

In [34]:
plt.figure(figsize=(10, 6))

review_scores = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin']

for score in review_scores:
    sns.scatterplot(x=score, y='price', data=subset_data, palette='pastel')
    plt.xlabel(f'Review Score ({score.replace("review_scores_", "")})')
    plt.ylabel('Listing Price')
    plt.title(f'Impact of {score} on Listing Price')
    plt.tight_layout()
    plt.show()


/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/1633087805.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Histograms show us the frequency distribution of the given variable

In [35]:

plt.figure(figsize=(10, 6))

review_scores = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin']

for score in review_scores:
    sns.histplot(x=score, data=subset_data, bins=10, palette='pastel')
    plt.xlabel(f'Review Score ({score.replace("review_scores_", "")})')
    plt.ylabel('Count')
    plt.title(f'Distribution of {score}')
    plt.tight_layout()
    plt.show()


/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/301942357.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Analysis of summary statstics

Correlation analysis - using Spearmans rank
Spearman's rank correlation is a way to see if there's a pattern between these two factors (in this case it will be price and a review score)
The stronger the correlation (closer to 1 or -1), the more closely review scores and prices are connected


In [36]:
# library needed for spearmans rank
from scipy.stats import spearmanr

spearman_corr, p_value = spearmanr(subset_data)

spearman_corr, p_value

(array([[ 1.        , -0.00249532, -0.03312067,  0.02690851, -0.04873188,
         -0.04677633,  0.13709622, -0.14556721],
        [-0.00249532,  1.        ,  0.72843649,  0.68304559,  0.58516083,
          0.61943917,  0.44517048,  0.70609842],
        [-0.03312067,  0.72843649,  1.        ,  0.62221426,  0.59373291,
          0.60645734,  0.42610174,  0.65170658],
        [ 0.02690851,  0.68304559,  0.62221426,  1.        ,  0.47020641,
          0.4785166 ,  0.36416713,  0.57820758],
        [-0.04873188,  0.58516083,  0.59373291,  0.47020641,  1.        ,
          0.67544983,  0.39614676,  0.52656025],
        [-0.04677633,  0.61943917,  0.60645734,  0.4785166 ,  0.67544983,
          1.        ,  0.3955357 ,  0.54930209],
        [ 0.13709622,  0.44517048,  0.42610174,  0.36416713,  0.39614676,
          0.3955357 ,  1.        ,  0.43251218],
        [-0.14556721,  0.70609842,  0.65170658,  0.57820758,  0.52656025,
          0.54930209,  0.43251218,  1.        ]]),
 array([[0.000

In [37]:
#Visualisation of Spearmans rank using heatmap we do this so we can look at all the different type of review scores
plt.figure(figsize=(8, 6))
sns.heatmap(subset_data.corr(method='spearman'), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title("Spearman's Rank Correlation Matrix")
plt.show()



/var/folders/26/p92lw58s64l1zlyt8yg13ztc0000gn/T/ipykernel_15048/1284917966.py:5: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


Analysis of spearmann rank

Linear regression is a commonly used statistical technique for modeling the relationship between one or more independent variables (predictors) and a continuous dependent variable (outcome).

multiple linear regression is more appropriate than simple linear regression because we are interested in assessing the combined impact of multiple factors (review scores for cleanliness, location, and communication) on listing prices. By using multiple linear regression, we can analyze how these review scores collectively influence listing prices while controlling for other variables. This approach allows for a more comprehensive understanding of the relationship between review scores and prices, accounting for potential interactions or confounding effects between predictors.
